In [1]:
from dask import dataframe as dd
import pandas as pd

In [18]:
rec_abx_sm_pct = 60
rec_abx_sm = 0

In [19]:
import dask.dataframe as dd
import pandas as pd
df = dd.read_parquet('D:/isaac/processado/remuneracoes_lv2.parquet').compute()
df = df[df.dataExtracao.eq('2019-12-09')]
df['dataEntradaRequerimento'] = df.dataEntradaRequerimento.astype('datetime64[ns]')
df = df.set_index('dataEntradaRequerimento')
df['grupo'] = df.apply(lambda r: 'flex' if r.rec_abx_sm <= rec_abx_sm and r.rec_abx_sm_pct >= 50/rec_abx_sm_pct else 'não Flex', axis=1)
ndf = df.pivot_table(index='dataEntradaRequerimento', columns=['grupo','resultado_simulacao'], values='idOrigin', aggfunc='count').fillna(0)
ndf.columns = ['flexiveis_com_direito','flexiveis_sem_direito','residual_com_direito','residual_sem_direito']
ndf = ndf.groupby(pd.Grouper(freq='M')).sum()
ndf['residual'] = ndf.residual_com_direito + ndf.residual_sem_direito
ndf['mes'] = ndf.index.month
ndf['ano'] = ndf.index.year
ndf

,flexiveis_com_direito,flexiveis_sem_direito,residual_com_direito,residual_sem_direito,residual,mes,ano
dataEntradaRequerimento,,,,,,,
2017-11-30,0.0,0.0,1.0,0.0,1.0,11,2017
2017-12-31,0.0,0.0,0.0,0.0,0.0,12,2017
2018-01-31,6.0,5.0,29.0,40.0,69.0,1,2018
2018-02-28,5.0,11.0,53.0,65.0,118.0,2,2018
2018-03-31,16.0,15.0,97.0,112.0,209.0,3,2018
2018-04-30,7.0,14.0,65.0,80.0,145.0,4,2018
2018-05-31,29.0,42.0,176.0,218.0,394.0,5,2018
2018-06-30,55.0,95.0,285.0,371.0,656.0,6,2018
2018-07-31,139.0,38.0,527.0,185.0,712.0,7,2018


In [17]:
df = dd.read_parquet('D:/isaac/processado/remuneracoes_lv2.parquet')
df['dataEntradaRequerimento'] = dd.to_datetime(df['dataEntradaRequerimento'])
df['dataExtracao'] = dd.to_datetime(df['dataExtracao'])
df = df[df['dataExtracao'] == df['dataExtracao'].max()]
df = df.compute()
df = df.sort_values('dataEntradaRequerimento')
#print(df.columns)
print('Max valor salário mínimo abaixo do mínimo: ', df['rec_abx_sm_pct'].max())
df.index = df.dataEntradaRequerimento
grouped = df.groupby([df.index.month, df.index.year])['idOrigin'].nunique()
new_index = [str(year)+'-'+str(month).zfill(2) for month, year in grouped.index]
grouped.index = new_index
grouped = grouped.sort_index()
print('Original')
print(grouped)
residual = {'x': grouped.index.astype(str).values.tolist(), 'y': grouped.values.tolist()}
if rec_abx_sm_pct is not None and rec_abx_sm is not None:
    df = df[df.dataExtracao.eq('2019-12-09')]
    df['dataEntradaRequerimento'] = df.dataEntradaRequerimento.astype('datetime64[ns]')
    df = df.set_index('dataEntradaRequerimento')
    df['grupo'] = df.apply(lambda r: 'flex' if r.rec_abx_sm <= rec_abx_sm and r.rec_abx_sm_pct >= 50/rec_abx_sm_pct else 'não Flex', axis=1)
    ndf = df.pivot_table(index='dataEntradaRequerimento', columns=['grupo','resultado_simulacao'], values='idOrigin', aggfunc='count').fillna(0)
    ndf.columns = ['flexiveis_com_direito','flexiveis_sem_direito','residual_com_direito','residual_sem_direito']
    ndf = ndf.groupby(pd.Grouper(freq='M')).sum()
    ndf['residual'] = ndf.residual_com_direito + ndf.residual_sem_direito
    ndf['mes'] = ndf.index.month
    ndf['ano'] = ndf.index.year
    competencias = ndf.apply(lambda x: x['ano'].astype(int).astype(str)+'-'+x['mes'].astype(int).astype(str), axis=1).values.tolist()
    nao_concedidos = {'x': competencias, 'y': ndf.flexiveis_sem_direito.astype(int).values.tolist()}
    print("não concedidos:\n", nao_concedidos)
    concedidos = {'x': competencias, 'y': ndf.flexiveis_com_direito.astype(int).values.tolist()}
    print("concedidos:\n", concedidos)
    residual = {'x': competencias, 'y': ndf.residual.astype(int).values.tolist()}
    print("residual:\n", residual)
else:
    nao_concedidos = {'x': [], 'y': []}
    concedidos = {'x': [], 'y': []}
ndf

Max valor salário mínimo abaixo do mínimo:  112.81954273271637
Original
2017-11       1
2018-01      80
2018-02     134
2018-03     240
2018-04     166
2018-05     465
2018-06     806
2018-07     889
2018-08    1323
2018-09    1111
2018-10    1808
2018-11    2071
2018-12    1481
2019-01    1598
2019-02    2182
2019-03    2183
2019-04    2685
2019-05    2174
2019-06    1477
2019-07    1728
2019-08    2184
2019-09    2820
2019-10    3842
2019-11    1405
Name: idOrigin, dtype: int64
não concedidos:
 {'x': ['2017-11', '2017-12', '2018-1', '2018-2', '2018-3', '2018-4', '2018-5', '2018-6', '2018-7', '2018-8', '2018-9', '2018-10', '2018-11', '2018-12', '2019-1', '2019-2', '2019-3', '2019-4', '2019-5', '2019-6', '2019-7', '2019-8', '2019-9', '2019-10', '2019-11'], 'y': [0, 0, 6, 18, 18, 17, 49, 113, 44, 165, 134, 204, 248, 161, 149, 210, 303, 296, 201, 260, 358, 343, 347, 674, 278]}
concedidos:
 {'x': ['2017-11', '2017-12', '2018-1', '2018-2', '2018-3', '2018-4', '2018-5', '2018-6', '2018-7', 

,flexiveis_com_direito,flexiveis_sem_direito,residual_com_direito,residual_sem_direito,residual,mes,ano
dataEntradaRequerimento,,,,,,,
2017-11-30,0.0,0.0,1.0,0.0,1.0,11,2017
2017-12-31,0.0,0.0,0.0,0.0,0.0,12,2017
2018-01-31,6.0,6.0,29.0,39.0,68.0,1,2018
2018-02-28,6.0,18.0,52.0,58.0,110.0,2,2018
2018-03-31,18.0,18.0,95.0,109.0,204.0,3,2018
2018-04-30,8.0,17.0,64.0,77.0,141.0,4,2018
2018-05-31,36.0,49.0,169.0,211.0,380.0,5,2018
2018-06-30,65.0,113.0,275.0,353.0,628.0,6,2018
2018-07-31,157.0,44.0,509.0,179.0,688.0,7,2018


In [14]:
import plotly
import chart_studio.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

plotly.offline.init_notebook_mode(connected=True)

data = [go.Bar(x=residual['x'], y=residual['y'], name="Residual", marker=dict(color='#37536D')),
       go.Bar(x=nao_concedidos['x'], y=nao_concedidos['y'], name="Não Concedidos", marker=dict(color='#ea4141')),
       go.Bar(x=concedidos['x'], y=concedidos['y'], name="Concedidos", marker=dict(color='#16cc77'))]

layout = dict(
    title='Nº de Entrada de Requerimentos',
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1,
                     label='1m',
                     step='month',
                     stepmode='backward'),
                dict(count=6,
                     label='6m',
                     step='month',
                     stepmode='backward'),
                dict(step='all',
                     label='Tudo',)
            ])
        ),
        rangeslider=dict(
            visible = True
        ),
        type='date',
        
    ),
    barmode='stack'
)

fig = dict(data=data, layout=layout)

plotly.offline.iplot(fig)